In [1]:
%load_ext sql
%sql sqlite://

'Connected: @None'

# String Expressions

String values can be combined to form longer strings using the concatenation `||` operator. 

In [3]:
%%sql
select "hello," || "world";

 * sqlite://
Done.


"""hello,"" || ""world"""
"hello,world"


Basic string manipulation is built into SQL, but differs from Python.

Let's say we have a table called `phrase` that has contains a column `s` with a value `"hello, world"`.

In [5]:
%%sql
create table phrase as select "hello, world" as s;

 * sqlite://
Done.


[]

In [6]:
%%sql
select * from phrase

 * sqlite://
Done.


s
"hello, world"


An example of a string manipulation in SQL is as the following,

In [7]:
%%sql
select substr(s, 4, 2) || substr(s, instr(s, " ")+1, 1) from phrase;

 * sqlite://
Done.


"substr(s, 4, 2) || substr(s, instr(s, "" "")+1, 1)"
low


From the string manipulation statement above, we obtain the string `"low"`! How does this work?

`substr(s, 4, 2)` means to take the substring of `s` starting from position 4 with a length 2. This is how SQL obtained the letter `lo`.

`||` is a concatenation

`substr(s, instr(s, " ") + 1, 1)` means to take the substring of `s` that starts at 1 element after the empty space `" "`, with a length of 1. This is how SQL obtained the letter `w`.

Strings can be used to represent structured values, but doing so is rarely a good idea. For example, if we try to treat each row as a linked list, 

In [2]:
%%sql
create table lists as select "one" as car, "two, three, four" as cdr;

 * sqlite://
Done.


[]

Above, the `"two, three, four"` uses a comma `,` as a delimiting character to separate one element with another (e.g. `two` with `three`). This is the meaning of using string to represent a structure. 

One possible way of obtaining data from this structure is as the following,

In [4]:
%%sql
select substr(cdr, 1, instr(cdr, ",") - 1) as cadr from lists;

 * sqlite://
Done.


cadr
two


It is certainly possible, but highly confusing. This is definitely not a good idea.

## Demo

Let's try combining strings together. We can start with creating the table `nouns`.

In [2]:
%%sql
create table nouns as
    select "dog" as phrase union
    select "cat" union
    select "bird";

 * sqlite://
Done.


[]

Now let's try getting whole sentences by joining the `nouns` table by itself. 

In [3]:
%%sql
select subject.phrase || " chased " || object.phrase
    from nouns as subject, nouns as object
    where subject.phrase <> object.phrase;

 * sqlite://
Done.


"subject.phrase || "" chased "" || object.phrase"
bird chased cat
bird chased dog
cat chased bird
cat chased dog
dog chased bird
dog chased cat


Above, we see that we can concatenate using the `||`.

Now let's try something more interesting! Let's create a table `ands`.

In [4]:
%%sql
create table ands as
    select first.phrase || " and " || second.phrase
    from nouns as first, nouns as second
    where first.phrase <> second.phrase;

 * sqlite://
Done.


[]

In [6]:
%%sql
select * from ands;

 * sqlite://
Done.


"first.phrase || "" and "" || second.phrase"
bird and cat
bird and dog
cat and bird
cat and dog
dog and bird
dog and cat


We want to use the table above; however, the name of the column looks messy! Let's recreate the new table and rename the column as `"phrase"`.

In [13]:
%%sql

drop table ands;

create table ands as
    select first.phrase || " and " || second.phrase as "phrase"
    from nouns as first, nouns as second
    where first.phrase <> second.phrase;


 * sqlite://
Done.
Done.


[]

In [14]:
%%sql
select * from ands;

 * sqlite://
Done.


phrase
bird and cat
bird and dog
cat and bird
cat and dog
dog and bird
dog and cat


In [15]:
%%sql
select subject.phrase || " chased " || object.phrase
    from ands as subject, ands as object
    where subject.phrase <> object.phrase;

 * sqlite://
Done.


"subject.phrase || "" chased "" || object.phrase"
bird and cat chased bird and dog
bird and cat chased cat and bird
bird and cat chased cat and dog
bird and cat chased dog and bird
bird and cat chased dog and cat
bird and dog chased bird and cat
bird and dog chased cat and bird
bird and dog chased cat and dog
bird and dog chased dog and bird
bird and dog chased dog and cat
